In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

df = pd.read_parquet('./data')
print(f'total dataset: {len(df)} rows')

## filters
# df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
# df = df[df['stabilized_spread'] > 0.2]  # nem tul nagy spread
# df = df[abs(df['last_price_delta_since_stabilized']) > 0.1]
# df = df[abs(df['last_price_delta_since_stabilized']) < 8]

# df = df[df['stabilized_amount_mean'] < 0.4]  # nem tul nagy tradek
# df = df[df['stabilized_nr_trades'] < 300]  # nem tul sok trade

df.sort_index(axis=1, inplace=True)

y = df['last_price_delta_since_stabilized']
X = df.drop('last_price_delta_since_stabilized', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)
X_train

total dataset: 23000 rows


,0_amount_mean,0_nr_trades,0_price_delta,0_spread,1_amount_mean,1_nr_trades,1_price_delta,1_spread,2_amount_mean,2_nr_trades,...,past_final_price_95,past_final_price_96,past_final_price_97,past_final_price_98,past_final_price_99,stabilized_amount_mean,stabilized_at_ms,stabilized_nr_trades,stabilized_spread,wave_direction
index,,,,,,,,,,,,,,,,,,,,,
0,0.0068,4.0,0.00,0.01,0.0082,5.0,0.00,0.01,0.0070,6.0,...,-1.44,3.61,2.13,0.71,0.0,0.0249,446,16.0,0.58,None
0,0.0210,32.0,1.36,2.90,0.0217,33.0,1.36,2.90,0.0219,34.0,...,-5.31,5.48,0.94,-3.05,0.0,0.0384,661,63.0,4.26,max
0,0.0203,7.0,0.35,0.73,0.0203,9.0,0.00,1.08,0.0334,16.0,...,1.95,-0.54,-1.00,0.76,0.0,0.0274,39,20.0,1.69,None
0,0.0194,3.0,0.40,0.01,0.0147,4.0,0.36,0.05,0.0136,6.0,...,-1.54,-1.95,-0.96,-1.75,0.0,0.0129,310,9.0,0.41,max
0,0.0414,57.0,0.27,0.98,0.0407,58.0,0.27,0.98,0.0394,60.0,...,1.34,1.21,1.36,1.60,0.0,0.0388,371,69.0,1.25,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0019,3.0,0.18,0.11,0.0014,4.0,0.17,0.12,0.0013,5.0,...,2.15,-0.02,1.19,0.51,0.0,0.0126,41,8.0,0.29,None
0,0.0446,214.0,0.20,5.72,0.0448,215.0,0.20,5.72,0.0455,216.0,...,1.95,-0.54,-1.00,0.76,0.0,0.0439,718,224.0,5.92,min
0,0.0118,8.0,0.40,0.30,0.0111,9.0,0.40,0.30,0.0605,33.0,...,2.78,2.70,3.13,4.78,0.0,0.0749,53,39.0,0.70,min


In [96]:
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
# from sklearn.ensemle import RandomForestRegressor

model = CatBoostRegressor(learning_rate=0.015, depth=7, loss_function='RMSE', random_state=0, verbose=False, iterations=1500)

pipeline = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    # ('scaler', StandardScaler()),
    # ('model', RandomForestRegressor(n_estimators=50, random_state=0))
    ('model', model)
])

pipeline.fit(X_train, y_train)
score = pipeline.score(X_valid, y_valid)
display(score)


0.46165769381852817

In [517]:
import catboost
w = catboost.MetricVisualizer('./models', subdirs=True)
w.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

boo


In [427]:
from sklearn.model_selection import GridSearchCV

do_grid_search = False

if do_grid_search:
    X = FeatureEngineer().transform(X)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

    parameters = {'depth': [7, 8],
                  'learning_rate': [0.015, 0.03, 0.04],
                  'iterations': [100, 1000, 1500]
                  }
    grid = GridSearchCV(estimator=model, param_grid=parameters, cv=2, n_jobs=-1)
    grid.fit(X_train, y_train)

    print(" Results from Grid Search ")
    print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
    print("\n The best score across ALL searched params:\n", grid.best_score_)
    print("\n The best parameters across ALL searched params:\n", grid.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.5933380986132425

 The best parameters across ALL searched params:
 {'depth': 7, 'iterations': 1500, 'learning_rate': 0.015}


In [97]:
from spreadsurfer import now_isoformat

save = True
if save:
    model.save_model(f'./models/{now_isoformat()}-score-{1000 * round(score, 3)}.cat')


In [94]:
input_from_log = """



"""
from timeit import timeit

mae = 0
count = 0
for line in input_from_log.split('\n'):
    if not line: continue
    count += 1
    json = '{' + line.split('{')[1]
    sample = pd.DataFrame([eval(json)])

    real = sample.last_price_delta_since_stabilized[0]
    print('expected: ', real)
    sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

    timeit(lambda: pipeline.predict(sample), number=1) * 1000
    guess = pipeline.predict(sample)[0]
    print('predict: ', guess)
    mae += abs(real - guess)
    print()
display(f'MAE: {mae / count}')

ZeroDivisionError: division by zero

In [438]:
sample = pd.DataFrame([
    {'0_amount_mean': 0.007, '0_nr_trades': 12.0, '0_price_delta': 0.91, '0_spread': 1.15, '1_amount_mean': 0.007, '1_nr_trades': 13.0, '1_price_delta': 0.91, '1_spread': 1.52, '2_amount_mean': 0.006, '2_nr_trades': 14.0,
     '2_price_delta': 0.91, '2_spread': 1.52, '3_amount_mean': 0.006, '3_nr_trades': 16.0, '3_price_delta': 0.69, '3_spread': 1.74, '4_amount_mean': 0.005, '4_nr_trades': 17.0, '4_price_delta': 0.69, '4_spread': 1.74,
     'last_price_delta_since_stabilized': -0.87, 'past_final_price_0': -0.11999999999898137, 'past_final_price_1': 0.25, 'past_final_price_2': -0.020000000000436557, 'past_final_price_3': -1.7400000000016007,
     'past_final_price_4': 0.06999999999970896, 'past_final_price_5': -2.7299999999995634, 'past_final_price_6': -3.9099999999998545, 'past_final_price_7': 3.360000000000582, 'past_final_price_8': -0.06000000000130967,
     'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.007, 'stabilized_at_ms': 369, 'stabilized_nr_trades': 18.0, 'stabilized_spread': 2.43, 'wave_direction': 'max'}
])

print('expected: ', sample.last_price_delta_since_stabilized.real())
sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

from timeit import timeit

timeit(lambda: pipeline.predict(sample), number=1) * 1000
print('predict: ', pipeline.predict(sample)[0])


expected:  -0.87
predict:  -0.9450021282696802


In [278]:
from glob import glob

cat_filename = glob('*.cat')[0]
cat_filename

model = CatBoostRegressor()
model.load_model(fname=cat_filename)
print(f'catboost model loaded from {cat_filename}')


catboost model loaded from 2023-01-21T20:37:02-score-773.0.cat
